# Session 2 modifying model in script


In [1]:
import numpy as np
import mujoco
import mediapy
import matplotlib.pyplot as plt
import time
import os

In [2]:
video_index = 0
def load_model_and_reset_data_from_xml(xml_string):
    mj_model = mujoco.MjModel.from_xml_string(xml_string)
    mj_data = mujoco.MjData(mj_model)

    mujoco.mj_resetData(mj_model, mj_data)
    return mj_data, mj_model
    # mj_data.joint('root').qvel = initial_qvel
def render(mj_data, mj_model, framerate=30, play_speed=1, num_frames=30 * 10, save_file_name = None):
    global video_index
    mj_renderer = mujoco.Renderer(mj_model)

    frames = []
    mujoco.mj_forward(mj_model, mj_data)
    for idx in range(num_frames):
        while mj_data.time * framerate / play_speed < idx:
            mujoco.mj_step(mj_model, mj_data)

        # mj_renderer.update_scene(mj_data, camera="fixed")
        mj_renderer.update_scene(mj_data, camera=0)
        frames.append(mj_renderer.render())

    mj_renderer.close()
    mediapy.show_video(frames, fps=framerate)
    video_index += 1
    if save_file_name is not None:
        path = os.path.join("result_videos", f"part2_{video_index:03d}_{save_file_name}")
        mediapy.write_video(path, frames, fps=framerate)

## modifying xml

In [3]:
import xml.etree.ElementTree as ET

In [4]:
mjcf_xml = """
<mujoco>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <contact>
        <exclude body1="coordinate_frame" body2="root_body"/>
    </contact>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -0.3">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <body name="root_body" pos="0 0 .2">
      <joint name="root"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="pendulum_1" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""

mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model, save_file_name=f"before_xml_modify.mp4")

root = ET.fromstring(mjcf_xml)
pendulum_1 = root.find(".//body[@name='pendulum_1']")

# Change color of pendulum_1's geom to green
pendulum_1_geom = pendulum_1.find('geom')
pendulum_1_geom.set('rgba', '0 1 0 1')

# Add a second pendulum
pendulum_2 = ET.SubElement(pendulum_1, 'body', {'name': 'pendulum_2', 'pos': '0 0 -.2'})
ET.SubElement(pendulum_2, 'joint')
ET.SubElement(pendulum_2, 'geom', {'fromto': '0 0 0 0 0 -.2', 'rgba': '1 0 0 1'})
mjcf_xml = ET.tostring(root, encoding='unicode')




mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model, save_file_name=f"after_xml_modify.mp4")



This browser does not support the video tag.


This browser does not support the video tag.


### merge two xml

In [5]:
xml1 = """
<mujoco>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <contact>
        <exclude body1="coordinate_frame" body2="root_body"/>
    </contact>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -0.3">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <body name="root_body" pos="0 0 .2">
      <joint name="root"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="pendulum_1" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""


xml2 = """
<mujoco>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>

    <body name="exo" pos="0 0 .2">
      <geom fromto="0 0 0 0 0 -.25" rgba="1 0 0 1"/>


      <body name="exo2" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 1 0 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""

import xml.etree.ElementTree as ET

# Parse XML strings into ElementTree objects
tree1 = ET.ElementTree(ET.fromstring(xml1))
tree2 = ET.ElementTree(ET.fromstring(xml2))
pendulum1 = tree1.find(".//body[@name='root_body']")
exo = tree2.find(".//body[@name='exo']")

# Change the position of exo body
exo.set('pos', '0.1 0 0')  # Move exo body to the right by 0.3 units


pendulum1.append(exo)
xml1_modified = ET.tostring(tree1.getroot(), encoding='unicode')

mj_data, mj_model = load_model_and_reset_data_from_xml(xml1)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model, save_file_name=f"xml1.mp4")


mj_data, mj_model = load_model_and_reset_data_from_xml(xml2)
# mj_data.joint('root').qvel = 10
render(mj_data, mj_model, save_file_name=f"xml2.mp4")



mj_data, mj_model = load_model_and_reset_data_from_xml(xml1_modified)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, save_file_name=f"xml1_modified.mp4")

mj_data, mj_model = load_model_and_reset_data_from_xml(xml1_modified)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model, save_file_name=f"xml1_modified_2.mp4")


This browser does not support the video tag.


This browser does not support the video tag.


This browser does not support the video tag.


## include in xml

## modifying model directly (Failed)

https://mujoco.readthedocs.io/en/stable/APIreference/APIfunctions.html#mjv-updatescene

In [6]:
mjcf_xml = """
<mujoco>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <contact>
        <exclude body1="coordinate_frame" body2="root_body"/>
    </contact>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -0.3">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <body name="root_body" pos="0 0 .2">
      <joint name="root"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="pendulum_1" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""





mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)


mj_data.joint('root').qvel = 10
render(mj_data, mj_model)


This browser does not support the video tag.


## include in xml

https://mujoco.readthedocs.io/en/stable/XMLreference.html#include

In [7]:
mjcf_xml = """
<mujoco>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>
    <include file="../../../../simhive/myo_sim/scene/myosuite_scene.xml"/>
</mujoco>
"""





# mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)


# mj_data.joint('root').qvel = 10
# render(mj_data, mj_model)
